$s$

In [1]:
import ipywidgets as widgets
out = widgets.Output()
with out:
    
    %bookmark 'model_dir'
    current_dir = %pwd
    while 'models' in current_dir:
        %cd ..
        current_dir = %pwd
    %bookmark 'base_dir'
    %cd source_code
    out.clear_output()
%run base
%run gui_v6
with out:
    %cd ..
    %cd models/fourbar
    out.clear_output()


In [74]:
m=model()
m.show()

## VEHICLE DYNAMICS MODELING AND SIMULATION TOOL

In [44]:
m.topology.edges

OutMultiEdgeView([(rbs_ground, rbs_link, 'jcs_sph')])

In [45]:
def equations_assembler(topology):
    
    edgelist = topology.edges(data='obj')
    nodelist = topology.nodes
    
    n_nodes = len(nodelist)
    n_edges = len(edgelist)
            
    jacobian = np.zeros((n_edges+n_nodes,n_nodes),dtype=np.object)
    jacobian.fill(None)
    
    equations = np.zeros((n_edges+n_nodes),dtype=np.object)
    
    vel_rhs = np.zeros((n_edges+n_nodes),dtype=np.object)
    acc_rhs = np.zeros((n_edges+n_nodes),dtype=np.object)
    
    reaction_vector = np.zeros((n_edges),dtype=np.object)
    reaction_indecies = np.zeros((n_edges),dtype=np.object)
    
    lamda_vector = np.zeros((n_edges+n_nodes),dtype=np.object)
    
    node_index = dict( (node,i) for i,node in enumerate(nodelist) )
    
    n7 = np.arange(7)
    for ei,e in enumerate(edgelist):
        (u,v) = e[:2]
        eo    = e[2]
        
        ui = node_index[u]
        vi = node_index[v]
        
        ui7 = (ui*7)+n7
        vi7 = (vi*7)+n7
        
        if jacobian[ui+n_edges,ui]==None: jacobian[ui+n_edges,ui] = (u.jac,ui7)
        if jacobian[vi+n_edges,vi]==None: jacobian[vi+n_edges,vi] = (v.jac,vi7)
                
        jacobian[ei,ui] = (eo.jacobian_i,ui7,vi7)
        jacobian[ei,vi] = (eo.jacobian_j,ui7,vi7)
        
        equations[ei] = (eo.equations,ui7,vi7)
        if equations[ui+n_edges]==0: equations[ui+n_edges]=(u.equations,ui7)
        if equations[vi+n_edges]==0: equations[vi+n_edges]=(v.equations,vi7)
        
        
        vel_rhs[ei] = ((eo.vel_rhs) if eo.nc==1 else np.zeros((eo.nc,1)))
        vel_rhs[ui+n_edges] = (np.array([[0]]) if u.nc==1 else np.zeros((7,1)))
        vel_rhs[vi+n_edges] = (np.array([[0]]) if v.nc==1 else np.zeros((7,1)))
        
        acc_rhs[ei] = (eo.acc_rhs,ui7,vi7)
        if acc_rhs[ui+n_edges]==0: acc_rhs[ui+n_edges]=(u.acc_rhs,ui7)
        if acc_rhs[vi+n_edges]==0: acc_rhs[vi+n_edges]=(v.acc_rhs,vi7)
            
        reaction_vector[ei] = (eo.reactions,ui7,vi7,eo.index)
        reaction_indecies[ei] = eo.reaction_index
        
        lamda_vector[ei] = eo.index
        if lamda_vector[ui+n_edges]==0: lamda_vector[ui+n_edges]=u.index
        if lamda_vector[vi+n_edges]==0: lamda_vector[vi+n_edges]=v.index
        
    #####################################################
    # Defining vectorized functions
    #####################################################
    
    def j_mapper(i,q):
        if len(i)==3:
            fun,bi,bj = i
            return fun(q[bi],q[bj])
        else:
            fun,bi = i
            return fun(q[bi])
    j_vectorized = np.vectorize(j_mapper,otypes=[np.object],excluded='q')
    
    
    def v_mapper(i):
        if isinstance(i,(int,np.ndarray)):
            return i
        else:
            return i()
    v_vectorized = np.vectorize(v_mapper,otypes=[np.object])
    
    def a_mapper(i,q,qd):
        if len(i)==3:
            fun,bi,bj = i
            return fun(q[bi],q[bj],qd[bi],qd[bj])
        else:
            fun,bi = i
            return fun(qd[bi])
    a_vectorized = np.vectorize(a_mapper,otypes=[np.object],excluded=('q','qd'))
    
    def r_mapper(i,q,lamda):
        fun,bi,bj,index = i
        return fun(q[bi],q[bj],lamda[index].values)
    r_vectorized = np.vectorize(r_mapper,otypes=[np.object],excluded=('q','lamda'))
    
    ####################################################################################
    ####################################################################################
    
    mass_matrix = np.zeros((n_nodes,n_nodes),dtype=np.object)
    mass_matrix.fill(None)
    
    gravitational_vector = np.zeros((n_nodes),dtype=np.object)
    centrifugal_vector   = np.zeros((n_nodes),dtype=np.object)
    inertiaforce_vector  = np.zeros((n_nodes),dtype=np.object)
    
    for n in nodelist:
        ni = node_index[n]
        ni7 = (ni*7)+n7
        
        mass_matrix[ni,ni]=(n.mass_matrix,ni7)
        
        gravitational_vector[ni]=n.gravity
        
        centrifugal_vector[ni]=(n.centrifugal,ni7)
        
        inertiaforce_vector[ni]=(n.inertia_force,ni7)
        
        
    def mass_mapper(i,q):
        fun,bi = i
        return fun(q[bi])
    mass_vectorized = np.vectorize(mass_mapper,otypes=[np.object],excluded='q')
    
    def centrifugal_mapper(i,q,qd):
        fun,bi = i
        return fun(q[bi],qd[bi])
    centrifugal_vectorized = np.vectorize(centrifugal_mapper,otypes=[np.object],excluded=('q','qd'))
    
    def gravity_mapper(i):
        return i()
    gravity_vectorized = np.vectorize(gravity_mapper,otypes=[np.object])
    
    
    system = pd.Series([jacobian,
                        jacobian.nonzero(),
                        equations, 
                        vel_rhs, 
                        acc_rhs,
                        reaction_vector,
                        reaction_indecies,
                        lamda_vector,
                        mass_matrix,
                        gravitational_vector,
                        centrifugal_vector,
                        inertiaforce_vector,
                        j_vectorized,
                        v_vectorized,
                        a_vectorized,
                        r_vectorized,
                        mass_vectorized,
                        centrifugal_vectorized,
                        gravity_vectorized],
                       index=['Jacobian','Jnzi','Ceq','Veq','Aeq','Rvec','Rind','Lind',
                              'MassMatrix','Gravity','Centrifugal','Inertia',
                              'Jf','Vf','Af','Rf','Mf','Cf','Gf'])
    
    return system


In [5]:
def jacobian_evaluator(jac_blocks,nzi,mapper,q,Id=None):
    A = jac_blocks.copy()
    A[nzi]=mapper(A[nzi],q=q)
    
    if Id is None:
        return sc.sparse.bmat(A,format='csc')
    else:
        J = sc.sparse.bmat(A,format='csc')
        return sc.sparse.bmat([[J],[Id]],format='csc')
    

In [6]:
def nr_kds(jac_blocks,equations_blocks,nzi,j_vectorized,guess,Id=None):
    
    A = jacobian_evaluator(jac_blocks,nzi,j_vectorized,guess,Id)
    
    b = equations_blocks.copy()
    bn = np.concatenate(j_vectorized(b[b.nonzero()],q=guess))
    
    if Id is not None:
        nqind=Id.shape[0]
        Ieq=np.zeros((nqind,1))
        bn = np.concatenate([bn,Ieq])
        
    delta_q = sc.sparse.linalg.spsolve(A,-bn)
    
    itr=0
    while np.linalg.norm(delta_q)>1e-5:
        
        guess=guess+delta_q
        
        if itr!=0 and itr%5==0:
            print('Recalculating Jacobian')
            A = jacobian_evaluator(jac_blocks,nzi,j_vectorized,guess,Id)
        bn = np.concatenate(j_vectorized(b[b.nonzero()],q=guess))
        if Id is not None:
            nqind=Id.shape[0]
            Ieq=np.zeros((nqind,1))
            bn = np.concatenate([bn,Ieq])
        delta_q = sc.sparse.linalg.spsolve(A,-bn)
        
        itr+=1
        
        if itr>200:
            print("Iterations exceded \n")
            break    
    
    return guess,jacobian_evaluator(jac_blocks,nzi,j_vectorized,guess,Id)


In [70]:
import scipy as sc

def coordinates_mapper(q):
    n=len(q)
    int2str=pd.Series(q.index,index=np.arange(0,n,1))
    str2int=pd.Series(np.arange(0,n,1),index=q.index)
    return int2str, str2int

def extract_ind(sparse_jac,q):
    mat=sparse_jac.A
    rows,cols=mat.shape
    permR=sc.linalg.lu(mat.T)[0]
    ind_cols=permR[:,rows:]
    maped=coordinates_mapper(q)[0]
    ind_coord=[maped[np.argmax(ind_cols[:,i])] for i in range(cols-rows) ]
    
    return ind_coord, ind_cols, permR

def assign_initial_conditions(q0,qd0,qind):
    q_initial  = list(q0[qind])
    qd_initial = list(qd0[qind])
    return q_initial+qd_initial



def dds(q0,qd0,system_equations,time_array):
    '''
    Dynamically Driven Systems Solver
    '''
    Cf = system_equations.Cf
    Gf = system_equations.Gf
    Mf  = system_equations.Mf
    Jf = eq_f = system_equations.Jf
    Af = system_equations.Af
    Vf = system_equations.Vf
    Rf = system_equations.Rf
    
    Cv = system_equations.Centrifugal
    Gv = system_equations.Gravity
    Mm = system_equations.MassMatrix
    Jm = system_equations.Jacobian
    Av = system_equations.Aeq
    Vv = system_equations.Veq
    Ev = system_equations.Ceq
    Rv = system_equations.Rvec
    Jnzi = system_equations.Jnzi
    
    # creating dataframes to hold the simulation results at each timestep
    # with initial conditions at t0
    position_df=pd.DataFrame(columns=q0.index)
    velocity_df=pd.DataFrame(columns=q0.index)
    acceleration_df=pd.DataFrame(columns=q0.index)
    lamda_df=pd.DataFrame(columns=np.concatenate(system_equations.Lind))
    JR_df=pd.DataFrame(columns=np.concatenate(system_equations.Rind))
    
    position_df.loc[0]=q0
    velocity_df.loc[0]=qd0
    
    # assigning initial conditions to the system NE equations
    M  = sc.sparse.block_diag(Mf(Mm.diagonal(),q=q0.values),format='csc')
    J  = jacobian_evaluator(Jm,Jnzi,Jf,q0.values)
    
    # Initiating coordinate partitioning
    qind=extract_ind(J,q0)
    qstr=qind[0]
    Ids=qind[1]
    perm_matrix = sc.sparse.csc_matrix(qind[2]).T
    qind_index=list(coordinates_mapper(q0)[1][qstr])
    dof = len(qstr)
    print('DOF : %s \n' %dof)
    print('Independent Coordinates are: %s with indices: %s \n'%(qstr,qind_index))
    
    
    Qv = np.concatenate(Cf(Cv,q=q0.values,qd=qd0.values))
    Qg = np.concatenate(Gf(Gv))
    Qt = (Qv+Qg)
    Qd = np.concatenate(Af(Av,q=q0.values,qd=qd0.values))
    
    
    # assembling the coefficient matrix and the rhs vector and solving for
    # system accelerations and lagrange multipliers
    coeff_matrix=sc.sparse.bmat([[M,J.T],[J,None]],format='csc')
    b_vector=np.concatenate([Qt,-Qd])
    x=sc.sparse.linalg.spsolve(coeff_matrix,b_vector)
    
    # updating the dataframes with the evaluated results
    qdd0n  = x[:len(q0)] # the first 7xnb elements in the x vector
    lamda0 = x[len(q0):] # the rest of elements in the x vector
    acceleration_df.loc[0]=qdd0n
    lamda_df.loc[0]=lamda0
    
    reaction=Rf(Rv,q=q0.values,lamda=lamda_df.loc[0])
    JR_df.loc[0]=np.concatenate(reaction)

    
    # Setting up the integrator function and the initial conditions
    
    def ssm(t,y,Mii,Mid,Qti,Cqi,lamda,qdd_d):
        
        v=list(y[len(y)//2:])
        vdot=sc.sparse.linalg.spsolve(Mii,(Qti.flatten()-(Cqi.T@lamda)-(Mid@qdd_d)))
        vdot=list(vdot)
        dydt=v+vdot
        return dydt
    

    Mp  = perm_matrix@M@perm_matrix.T
    Mii = Mp[-dof:,-dof:]
    Mid = Mp[-dof:,:-dof]

    Qtp = perm_matrix@Qt
    Qti = Qtp[-dof:]

    Cqp = J@perm_matrix.T
    Cqi = Cqp[:,-dof:]

    qdd_d = (perm_matrix@qdd0n)[:-dof]
    
    
    #return position_df,velocity_df,acceleration_df,JR_df, acc_k
    
    r=sc.integrate.ode(ssm).set_integrator('dop853')
    y0=assign_initial_conditions(q0,velocity_df.loc[0],qstr)
    r.set_initial_value(y0).set_f_params(Mii,Mid,Qti,Cqi,lamda0,qdd_d)
    
    # Setting up the time array to be used in integration steps and starting
    # the integration
    dtt=time_array[1]-time_array[0]
    for i,dt in enumerate(time_array):
        #print('time_step: '+str(i))
        #progress_bar(len(t),i)

        r.integrate(dt)
        print(r.y)
#        if not r.successful():
#            print("BREAKING SOLVER")
#            return position_df,velocity_df,acceleration_df,JR_df
        
        # creating the guess vector for the vector q as the values of the 
        # previous step and the value of newly evaluated independent coordinate
        guess=position_df.loc[i] + velocity_df.loc[i]*dtt + 0.5*acceleration_df.loc[i]*dtt**2
        guess[qstr]=r.y[:len(r.y)//2]
        
        # Evaluating the dependent vector q using newton raphson
        dependent=nr_kds(Jm,Ev,Jnzi,Jf,guess,Ids.T)
        position_df.loc[i+1]=dependent[0]
        Cq_new=dependent[1]
        
        # Calculating the system velocities
        vrhs=np.concatenate(Vf(Vv))
        vind=np.array(r.y[len(r.y)//2:]).reshape((len(r.y)//2,1))
        vrhs=np.concatenate([-vrhs,vind])
        velocity_df.loc[i+1]=sc.sparse.linalg.spsolve(Cq_new,vrhs)
        
        
        q=position_df.loc[i+1]
        qd=velocity_df.loc[i+1]
        
        # Evaluating the new coeff matrix bloks of the system NE equations
        M  = sc.sparse.block_diag(Mf(Mm.diagonal(),q=q.values),format='csc')
        J  = jacobian_evaluator(Jm,Jnzi,Jf,q.values)
        Qv = np.concatenate(Cf(Cv,q=q.values,qd=qd.values))
        Qg = np.concatenate(Gf(Gv))
        Qt = (Qv+Qg)
        Qd = np.concatenate(Af(Av,q=q.values,qd=qd.values))

        coeff_matrix=sc.sparse.bmat([[M,J.T],[J,None]],format='csc')
        b_vector=np.concatenate([Qt,-Qd])
        # Evaluating the acceleration and lagrange mult. vector
        x=sc.sparse.linalg.spsolve(coeff_matrix,b_vector)
        
        # updating the dataframes with the evaluated results
        qdd   = x[:len(q0)] # the first 7xnb elements in the x vector
        lamda = x[len(q0):] # the rest of elements in the x vector
        acceleration_df.loc[i+1]=qdd
        lamda_df.loc[i+1]=lamda
        reaction=Rf(Rv,q=q.values,lamda=lamda_df.loc[i+1])
        JR_df.loc[i+1]=np.concatenate(reaction)
        

        # Setting the ssm input parameters
        Mp = perm_matrix@M@perm_matrix.T
        Mii = Mp[-dof:,-dof:]
        Mid = Mp[-dof:,:-dof]

        Qtp = perm_matrix@Qt
        Qti = Qtp[-dof:]

        Cqp = J@perm_matrix.T
        Cqi = Cqp[:,-dof:]

        qdd_d = (perm_matrix@qdd0n)[:-dof]
        
        r.set_f_params(Mii,Mid,Qti,Cqi,lamda,qdd_d)
    
    
    return position_df,velocity_df,acceleration_df,JR_df


In [75]:
q0=pd.concat([i.q0 for i in m.topology.nodes])
qd0 = 0*q0
assembled  = equations_assembler(m.topology)
time_array = np.arange(0.001,1,1e-3)
tt=time.time()
d = dds(q0,qd0,assembled,time_array)
print(-tt+time.time())

DOF : 1 

Independent Coordinates are: ['rbs_link.z'] with indices: [9] 

[ -3.67847411e-03  -7.35694823e+00]
[ -0.0147139  -14.71389646]
[ -0.03310627 -22.07084468]
[ -0.05885559 -29.42779289]
[ -0.09196185 -36.78474104]
[ -0.13242507 -44.14168908]
[ -0.18024523 -51.49863692]
[ -0.23542234 -58.85558443]
[ -0.2979564  -66.21253144]
[ -0.3678474  -73.56947771]
[ -0.44509535 -80.92642295]
[ -0.52970025 -88.28336681]
[ -0.62166209 -95.64030884]
[  -0.72098087 -102.99724854]
[  -0.82765658 -110.3541853 ]
[  -0.94168923 -117.71111841]
[  -1.06307882 -125.06804707]
[  -1.19182533 -132.42497035]
[  -1.32792875 -139.78188723]
[  -1.4713891  -147.13879654]
[  -1.62220634 -154.49569699]
[  -1.78038049 -161.85258713]
[  -1.94591151 -169.2094654 ]
[  -2.11879941 -176.56633006]
[  -2.29904416 -183.92317921]
[  -2.48664576 -191.28001078]
[  -2.68160418 -198.63682253]
[  -2.88391939 -205.99361205]
[  -3.09359139 -213.35037672]
[  -3.31062013 -220.70711372]
[  -3.5350056  -228.06382005]
[  -3.76674776

In [8]:
q0=pd.concat([i.q0 for i in m.topology.nodes])
qd0 = 0*q0
assembled = equations_assembler(m.topology)

Cf = assembled.Cf
Gf = assembled.Gf
Mf  = assembled.Mf
Jf = eq_f = assembled.Jf
Af = assembled.Af
Vf = assembled.Vf
Rf = assembled.Rf

Cv = assembled.Centrifugal
Gv = assembled.Gravity
Mm = assembled.MassMatrix
Jm = assembled.Jacobian
Av = assembled.Aeq
Vv = assembled.Veq
Ev = assembled.Ceq
Rv = assembled.Rvec
Jnzi = assembled.Jnzi

# creating dataframes to hold the simulation results at each timestep
position_df=pd.DataFrame(columns=q0.index)
velocity_df=pd.DataFrame(columns=q0.index)
acceleration_df=pd.DataFrame(columns=q0.index)
lamda_df=pd.DataFrame(columns=np.concatenate(assembled.Lind))
JR_df=pd.DataFrame(columns=np.concatenate(assembled.Rind))

position_df.loc[0]=q0
velocity_df.loc[0]=qd0

# assigning initial conditions to the system NE equations
M  = sc.sparse.block_diag(Mf(Mm.diagonal(),q=q0.values),format='csc')
#M[10:,10:]=Jp
J  = jacobian_evaluator(Jm,Jnzi,Jf,q0.values)

# Initiating coordinate partitioning
qind=extract_ind(J,q0)
qstr=qind[0]
Ids=qind[1]
perm_matrix = sc.sparse.csc_matrix(qind[2]).T
qind_index=list(coordinates_mapper(q0)[1][qstr])
dof = len(qstr)
print('DOF : %s \n' %dof)
print('Independent Coordinates are: %s with indices: %s \n'%(qstr,qind_index))


Qv = np.concatenate(Cf(Cv,q=q0.values,qd=qd0.values))
Qg = np.concatenate(Gf(Gv))
Qt = (Qv+Qg)
Qd = np.concatenate(Af(Av,q=q0.values,qd=qd0.values))


# assembling the coefficient matrix and the rhs vector and solving for
# system accelerations and lagrange multipliers
coeff_matrix=sc.sparse.bmat([[M,J.T],[J,None]],format='csc')
b_vector=np.concatenate([Qt,-Qd])
x=sc.sparse.linalg.spsolve(coeff_matrix,b_vector)

# updating the dataframes with the evaluated results
qdd0n  = x[:len(q0)] # the first 7xnb elements in the x vector
lamda0 = x[len(q0):] # the rest of elements in the x vector
acceleration_df.loc[0]=qdd0n
lamda_df.loc[0]=lamda0
Qc = -J.T@lamda0

reaction=Rf(Rv,q=q0.values,lamda=lamda_df.loc[0])
JR_df.loc[0]=np.concatenate(reaction)

Mp  = perm_matrix@M@perm_matrix.T
Mii = Mp[-dof:,-dof:]
Mid = Mp[-dof:,:-dof]

Qtp = perm_matrix@Qt
Qti = Qtp[-dof:]

Cqp = J@perm_matrix.T
Cqi = Cqp[:,-dof:]

qdd_d = (perm_matrix@qdd0n)[:-dof]

DOF : 1 

Independent Coordinates are: ['rbs_link.z'] with indices: [9] 



In [10]:
ipy.display.display(pd.DataFrame(lamda_df))
ipy.display.display(pd.DataFrame(acceleration_df))
ipy.display.display(pd.DataFrame(JR_df))

,jcs_rev_eq0,jcs_rev_eq1,jcs_rev_eq2,jcs_rev_eq3,jcs_rev_eq4,rbs_ground_eq0,rbs_ground_eq1,rbs_ground_eq2,rbs_ground_eq3,rbs_ground_eq4,rbs_ground_eq5,rbs_ground_eq6,rbs_link_eq0
0,2.605442e-08,-2.307622e-08,4.747197e+07,-0.000025,0.000018,-2.605442e-08,2.307627e-08,-4.748178e+07,-4.272850e-22,0.000009,6.969567e-21,0.000061,-0.000028


,rbs_ground.x,rbs_ground.y,rbs_ground.z,rbs_ground.e0,rbs_ground.e1,rbs_ground.e2,rbs_ground.e3,rbs_link.x,rbs_link.y,rbs_link.z,rbs_link.e0,rbs_link.e1,rbs_link.e2,rbs_link.e3
0,0.0,-5.556470e-14,9.094947e-13,-0.0,-0.0,-0.0,-0.0,1.346328e-12,-1.192433e-12,-7356.948229,0.995388,2.403079,-0.995388,-2.403079


,jcs_rev_Fx,jcs_rev_Fy,jcs_rev_Fz,jcs_rev_Mx,jcs_rev_My,jcs_rev_Mz
0,-2.605442e-08,2.307622e-08,-4.747197e+07,0.000004,3.484784e-21,0.00003


In [120]:
Mp  = perm_matrix@M@perm_matrix.T
Mii = Mp[-dof:,-dof:]
Mid = Mp[-dof:,:-dof]
Mdd = Mp[:-dof,:-dof]
Mdi = Mp[:-dof,-dof:]

Qtp = perm_matrix@Qt
Qti = Qtp[-dof:]
Qtd = Qtp[:-dof]

Cqp = J@perm_matrix.T
Cqd = Cqp[:,:-dof]
Cqi = Cqp[:,-dof:]

Qd = np.concatenate(Af(Av,q=q0.values,qd=qd0.values))

H = -1*np.linalg.inv(Cqd.A)@Cqi
I = np.eye(dof)
B = np.concatenate([H,I])

qdd_d = (perm_matrix@qdd0n)[:-dof]

Mcab = Mii+(Mid@H)+H.T@(Mdi+Mdd@H)
Qcab = Qti + H.T@Qtd - (Mid+H.T@Mdd)@np.linalg.inv(Cqd.A)@Qd


Mcab.A,Mii.A

ipy.display.display(pd.DataFrame(Qcab))
ipy.display.display(pd.DataFrame(Mcab))
ipy.display.display(pd.DataFrame(Mii.A))
np.linalg.solve(Mcab,Qcab)

,0
0,-1.898452e+08
1,0.000000e+00
2,0.000000e+00


,0,1,2
0,25805.546945,4.908113e+03,1.603190
1,4908.112885,3.627604e+07,11849.232691
2,1.603190,1.184923e+04,25808.753325


,0,1,2
0,19352.210746,0.000000e+00,0.000000
1,0.000000,1.291195e+10,0.000000
2,0.000000,0.000000e+00,19352.210746


matrix([[ -7.35694823e+03],
        [  9.95387923e-01],
        [  1.66189950e-13]])

In [121]:
q0

rbs_ground.x        0.000000
rbs_ground.y        0.000000
rbs_ground.z        0.000000
rbs_ground.e0       1.000000
rbs_ground.e1       0.000000
rbs_ground.e2       0.000000
rbs_ground.e3       0.000000
rbs_link.x       1000.000000
rbs_link.y          0.000000
rbs_link.z          0.000000
rbs_link.e0        -0.270598
rbs_link.e1         0.653281
rbs_link.e2        -0.270598
rbs_link.e3         0.653281
dtype: float64

In [122]:
m_link = sc.sparse.bmat([[19352.210746*np.eye(3),None],
                         [None,M[10:,10:]]])
m_ground = M[0:7,0:7]
Mass_Matrix = sc.sparse.bmat([[m_ground,None],
                              [None,m_link]],format='csc')
j_mount = np.eye(7)
j_link  = 2*np.array([0,0,0,-0.270598,0.653281,-0.270598,0.653281])
jacobian = sc.sparse.bmat([[j_mount,None],
                           [None ,j_link]],format='csc')
Coeff_matrix = sc.sparse.bmat([[Mass_Matrix,jacobian.T],
                               [jacobian,None]])

accRHS = np.zeros((8,))
external_forces = np.zeros((14,)); external_forces[9]=19352.210746*9810
b_vector = np.concatenate([external_forces,accRHS])

np.linalg.solve(Coeff_matrix.A,b_vector)

array([    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,  9810.,     0.,     0.,     0.,     0.,    -0.,    -0.,
          -0.,     0.,    -0.,    -0.,    -0.,     0.])

time_array = np.arange(5e-4,0.5,5e-4)
d= dds(q0,0*q0,assembled,time_array)

In [76]:
def plots(i):
    plt.figure(figsize=(10,20))

    plt.subplot(611)
    plt.title('z')
    plt.plot(time_array[:i],d[0]['rbs_link.z'][:i])
    plt.grid()

    plt.subplot(612)
    plt.title('x')
    plt.plot(time_array[:i],d[0]['rbs_link.x'][:i])
    
    plt.subplot(613)
    plt.title('e0')
    plt.plot(time_array[:i],d[0]['rbs_link.e0'][:i])
    
    plt.subplot(614)
    plt.title('e1')
    plt.plot(time_array[:i],d[0]['rbs_link.e1'][:i])
    
    plt.subplot(615)
    plt.title('e2')
    plt.plot(time_array[:i],d[0]['rbs_link.e2'][:i])
    
    plt.subplot(616)
    plt.title('e3')
    plt.plot(time_array[:i],d[0]['rbs_link.e3'][:i])

    plt.grid()
    plt.show()

widgets.interact(plots,i=widgets.IntSlider(value=1,max=len(time_array)),step=1)

interactive(children=(IntSlider(value=1, description='i', max=999), Output()), _dom_classes=('widget-interact'…

<function __main__.plots>

In [50]:
def plots(i):
    plt.figure(figsize=(10,20))

    plt.subplot(611)
    plt.title('z')
    plt.plot(time_array[:i],d[1]['rbs_link.z'][:i])

    plt.subplot(612)
    plt.title('x')
    plt.plot(time_array[:i],d[1]['rbs_link.x'][:i])
    
    plt.subplot(613)
    plt.title('e0')
    plt.plot(time_array[:i],d[1]['rbs_link.e0'][:i])
    
    plt.subplot(614)
    plt.title('e1')
    plt.plot(time_array[:i],d[1]['rbs_link.e1'][:i])
    
    plt.subplot(615)
    plt.title('e2')
    plt.plot(time_array[:i],d[1]['rbs_link.e2'][:i])
    
    plt.subplot(616)
    plt.title('e3')
    plt.plot(time_array[:i],d[1]['rbs_link.e3'][:i])

    plt.grid()
    plt.show()

widgets.interact(plots,i=widgets.IntSlider(value=1,max=len(time_array)))

interactive(children=(IntSlider(value=1, description='i', max=2499), Output()), _dom_classes=('widget-interact…

<function __main__.plots>

In [51]:

def plots(i):
    plt.figure(figsize=(10,20))

    plt.subplot(611)
    plt.title('z')
    plt.plot(time_array[:i],d[2]['rbs_link.z'][:i])

    plt.subplot(612)
    plt.title('x')
    plt.plot(time_array[:i],d[2]['rbs_link.x'][:i])
    
    plt.subplot(613)
    plt.title('e0')
    plt.plot(time_array[:i],d[2]['rbs_link.e0'][:i])
    
    plt.subplot(614)
    plt.title('e1')
    plt.plot(time_array[:i],d[2]['rbs_link.e1'][:i])
    
    plt.subplot(615)
    plt.title('e2')
    plt.plot(time_array[:i],d[2]['rbs_link.e2'][:i])
    
    plt.subplot(616)
    plt.title('e3')
    plt.plot(time_array[:i],d[2]['rbs_link.e3'][:i])

    plt.grid()
    plt.show()

widgets.interact(plots,i=widgets.IntSlider(value=1,max=len(time_array)))

interactive(children=(IntSlider(value=1, description='i', max=2499), Output()), _dom_classes=('widget-interact…

<function __main__.plots>